In [1]:
# Data manipulation
import numpy as np
import pandas as pd
from math import *
import seaborn as sns
import networkx as nx
import os
import scipy.stats as stats

# Visualization.
import matplotlib.pyplot as plt

# Saving models
from datetime import datetime
import joblib

# Display all columns
pd.set_option('display.max_columns', 150,
             'display.max_rows', 150)

# ML
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, RepeatedKFold, train_test_split, cross_validate, cross_val_score 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, median_absolute_error#, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor


import requests, json
from sqlalchemy import create_engine

#### Load helper functions

In [2]:
%run ./functions.ipynb

In [3]:
path0 = os.getcwd()

In [4]:
full = pd.read_csv(os.path.join(path0, "Full.csv"))

In [5]:
full.head(5)

,Unnamed: 0,order_id,lat_destination,lng_destination,promised_time,on_demand,shopper_id,store_branch_id,total_minutes,quantity_UN,quantity_KG,is_more_UN,UN_plus_KG,UN_mult_KG,seniority,found_rate,picking_speed,accepted_rate,rating,store_id,lat_origin,lng_origin,Hour,Month,Day_of_Week,Year,Date,period,distance_havesine,distance_car,weight_car,duration,city_origin,state_origin,county_origin,neighbourhood_origin,city_destiny,state_destiny,county_destiny,neighbourhood_destiny,same_city,same_state,same_county,same_neighbourhood,path_city,path_state,path_county,shoppers_number,store_branch_number
0,0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,16.0,2.756,1.0,18.756,44.096,6c90661e6d2c7579f5ce337c3391dbb9,0.9024,1.30,0.92,4.76,c4ca4238a0b923820dcc509a6f75849b,-33.485280,-70.579250,20,10,4,2019,18,night,1.823597,3367.1,380.8,380.8,Macul,Región Metropolitana de Santiago,Provincia de Santiago,Villa Universidad Católica,Peñalolén,Región Metropolitana de Santiago,Provincia de Santiago,Conjunto San Luis,0.0,1.0,1.0,0.0,Macul_X_Peñalolén,Región Metropolitana de Santiago_X_Región Metr...,Provincia de Santiago_X_Provincia de Santiago,7698,7698
1,1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,11.0,0.000,1.0,11.000,0.000,41dc7c9e385c4d2b6c1f7836973951bf,0.7610,2.54,0.92,4.96,c4ca4238a0b923820dcc509a6f75849b,-33.441246,-70.535450,1,10,5,2019,19,dawn,1.935026,2373.9,229.1,229.1,undefined,Región Metropolitana de Santiago,Provincia de Santiago,La Reina,undefined,Región Metropolitana de Santiago,Provincia de Santiago,La Reina,0.0,1.0,1.0,1.0,undefined_X_undefined,Región Metropolitana de Santiago_X_Región Metr...,Provincia de Santiago_X_Provincia de Santiago,7698,7698
2,2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,18.0,0.000,1.0,18.000,0.000,50e13ee63f086c2fe84229348bc91b5b,0.8313,2.57,0.76,4.92,c4ca4238a0b923820dcc509a6f75849b,-33.008213,-71.545615,14,10,5,2019,19,afternoon,2.358128,2930.5,298.2,298.2,Viña del Mar,Región de Valparaíso,Provincia de Valparaíso,Población Británica,Viña del Mar,Región de Valparaíso,Provincia de Valparaíso,Población Naval Las Salinas,1.0,1.0,1.0,0.0,Viña del Mar_X_Viña del Mar,Región de Valparaíso_X_Región de Valparaíso,Provincia de Valparaíso_X_Provincia de Valparaíso,7698,7698
3,3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742,1.0,0.000,1.0,1.000,0.000,41dc7c9e385c4d2b6c1f7836973951bf,0.8776,2.80,0.96,4.76,f718499c1c8cef6730f9fd03c8125cab,-33.355258,-70.537787,21,10,4,2019,18,night,3.820244,5632.1,3826.0,549.3,Lo Barnechea,Región Metropolitana de Santiago,Provincia de Santiago,Las Pataguas,Lo Barnechea,Región Metropolitana de Santiago,Provincia de Santiago,Alpes Suizos,1.0,1.0,1.0,0.0,Lo Barnechea_X_Lo Barnechea,Región Metropolitana de Santiago_X_Región Metr...,Provincia de Santiago_X_Provincia de Santiago,51,51
4,4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,2019-10-19 20:00:00+00:00,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822,91.0,6.721,1.0,97.721,611.611,50e13ee63f086c2fe84229348bc91b5b,0.7838,2.40,0.96,4.96,c4ca4238a0b923820dcc509a6f75849b,-33.386547,-70.568075,20,10,5,2019,19,night,1.894474,2939.7,223.8,223.8,Vitacura,Región Metropolitana de Santiago,Provincia de Santiago,NaN,Las Condes,Región Metropolitana de Santiago,Provincia de Santiago,NaN,0.0,1.0,1.0,0.0,Vitacura_X_Las Condes,Región Metropolitana de Santiago_X_Región Metr...,Provincia de Santiago_X_Provincia de Santiago,7698,7698


In [6]:
full.columns

Index(['Unnamed: 0', 'order_id', 'lat_destination', 'lng_destination',
       'promised_time', 'on_demand', 'shopper_id', 'store_branch_id',
       'total_minutes', 'quantity_UN', 'quantity_KG', 'is_more_UN',
       'UN_plus_KG', 'UN_mult_KG', 'seniority', 'found_rate', 'picking_speed',
       'accepted_rate', 'rating', 'store_id', 'lat_origin', 'lng_origin',
       'Hour', 'Month', 'Day_of_Week', 'Year', 'Date', 'period',
       'distance_havesine', 'distance_car', 'weight_car', 'duration',
       'city_origin', 'state_origin', 'county_origin', 'neighbourhood_origin',
       'city_destiny', 'state_destiny', 'county_destiny',
       'neighbourhood_destiny', 'same_city', 'same_state', 'same_county',
       'same_neighbourhood', 'path_city', 'path_state', 'path_county',
       'shoppers_number', 'store_branch_number'],
      dtype='object')

    Using the copernicus era5 data I crated a data base, using R, with temperature and precipitation to each county region in each day. Please, to see the R script.

In [7]:
temp = pd.read_csv(os.path.join(path0, "chile_temp.csv"),  index_col=0) # temperature
pre = pd.read_csv(os.path.join(path0, "chile_pre.csv"), index_col=0) # precipitation

In [8]:
full = pd.merge(full, temp, how='left', on=['county_origin','Date'])
full = pd.merge(full, pre, how='left', on=['county_origin','Date'])

    To treat the missing values I applied the zscore in the 'quantity_KG','quantity_UN', 'distance_car', 'distance_havesine' columns

In [9]:
full = zscore(full, cols = ['quantity_KG','quantity_UN', 'distance_car', 'distance_havesine'])

In [10]:
full.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9438 entries, 0 to 9977
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             9438 non-null   int64  
 1   order_id               9438 non-null   object 
 2   lat_destination        9438 non-null   float64
 3   lng_destination        9438 non-null   float64
 4   promised_time          9438 non-null   object 
 5   on_demand              9438 non-null   bool   
 6   shopper_id             9438 non-null   object 
 7   store_branch_id        9438 non-null   object 
 8   total_minutes          7558 non-null   float64
 9   quantity_UN            9438 non-null   float64
 10  quantity_KG            9438 non-null   float64
 11  is_more_UN             9438 non-null   float64
 12  UN_plus_KG             9438 non-null   float64
 13  UN_mult_KG             9438 non-null   float64
 14  seniority              9438 non-null   object 
 15  foun

In [11]:
full.isnull().sum()

Unnamed: 0                  0
order_id                    0
lat_destination             0
lng_destination             0
promised_time               0
on_demand                   0
shopper_id                  0
store_branch_id             0
total_minutes            1880
quantity_UN                 0
quantity_KG                 0
is_more_UN                  0
UN_plus_KG                  0
UN_mult_KG                  0
seniority                   0
found_rate                  0
picking_speed               0
accepted_rate               0
rating                      0
store_id                    0
lat_origin                  0
lng_origin                  0
Hour                        0
Month                       0
Day_of_Week                 0
Year                        0
Date                        0
period                      0
distance_havesine           0
distance_car                0
weight_car                  0
duration                    0
city_origin                 0
state_orig

In [12]:
var_cat = ['on_demand', 'seniority', 'Hour', 'Day_of_Week', 'period',
           'county_origin', 'county_destiny', 'state_origin', 'state_destiny',
           'city_origin', 'city_destiny',
           'path_city',
           'path_state', 
           'path_county',
           'same_city', 'same_state', 'same_county',
           'same_neighbourhood'
          ]

var_num = ['quantity_UN', 'quantity_KG', 'UN_plus_KG', 'UN_mult_KG',
           'distance_havesine',
           'found_rate', 'picking_speed','accepted_rate', 'rating',
           'distance_car', 
           'duration',
           'shoppers_number',
           'store_branch_number', 'temperature', 'precipitation'
          ]

    Transform the categorical variables in dummies

In [13]:
dft = pd.get_dummies(full[var_cat], columns= var_cat, drop_first=True)
cat_var = dft.columns

In [14]:
full = pd.get_dummies(full, columns = var_cat)

    Here I drop the variable with more than 0.8 of correlation.

In [15]:
corr_var = corrX_orig(full[list(cat_var)+list(var_num)], cut = 0.8)

C:\Users\AURELI~1.PAI\AppData\Local\Temp/ipykernel_17240/1833520548.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  up = corr_mtx.where(np.triu(np.ones(corr_mtx.shape), k=1).astype(np.bool))


In [16]:
target = ['total_minutes', 'order_id']
full = full[list(set(full[list(cat_var) + list(var_num)].columns) - set(corr_var)) + target]

In [17]:
var_num = list(set(var_num) - set(corr_var))

### Out of Sample

In [18]:
ofs = full[full.total_minutes.isnull()]

In [19]:
ofs.head(3)

,city_origin_La Serena,city_origin_Conchalí,Hour_1,city_origin_Viña del Mar,city_origin_Colina,Hour_3,city_destiny_San Pedro de la Paz,Hour_21,city_destiny_La Serena,city_origin_Valparaíso,Hour_19,distance_havesine,city_origin_Las Condes,city_destiny_undefined,city_origin_Vitacura,city_origin_Recoleta,city_origin_Concepción,city_origin_Coquimbo,city_destiny_La Granja,city_destiny_Lo Espejo,city_origin_San Pedro de la Paz,city_destiny_Vitacura,county_destiny_Provincia de Maipo,city_destiny_Providencia,Hour_15,seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f,county_destiny_Provincia de Cordillera,city_destiny_Quinta Normal,city_destiny_San Ramón,city_destiny_Talcahuano,city_destiny_Lo Prado,Hour_2,city_origin_undefined,city_destiny_Renca,city_destiny_La Pintana,Hour_23,city_origin_Huechuraba,city_destiny_La Florida,shoppers_number,Day_of_Week_5,city_origin_Santiago,Hour_12,county_origin_Provincia de Santiago,seniority_50e13ee63f086c2fe84229348bc91b5b,city_origin_Estación Central,Day_of_Week_6,city_destiny_Cerro Navia,city_origin_Ñuñoa,temperature,seniority_6c90661e6d2c7579f5ce337c3391dbb9,city_origin_San Miguel,county_origin_Provincia de Cordillera,city_destiny_Concepción,Hour_17,on_demand_True,Hour_13,city_destiny_Macul,city_destiny_Coquimbo,city_destiny_Peñalolén,city_origin_Providencia,city_origin_Pudahuel,city_origin_Peñalolén,Hour_20,quantity_KG,city_destiny_San Joaquín,city_destiny_San Miguel,city_destiny_Las Condes,Hour_22,found_rate,city_destiny_Pudahuel,city_destiny_Ñuñoa,city_origin_Talcahuano,city_destiny_Estación Central,city_destiny_Huechuraba,city_destiny_Recoleta,Hour_18,city_destiny_Santiago,rating,city_destiny_Conchalí,accepted_rate,Hour_11,period_dawn,city_destiny_Lo Barnechea,city_destiny_El Bosque,city_destiny_La Cisterna,city_destiny_Valparaíso,same_county_1.0,Hour_14,same_city_1.0,Hour_16,same_neighbourhood_1.0,city_origin_Macul,city_origin_San Joaquín,period_night,quantity_UN,city_destiny_Independencia,picking_speed,total_minutes,order_id
2,0,0,0,1,0,0,0,0,0,0,0,2.358128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7698,1,0,0,0,1,0,0,0,0,285.531514,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0.8313,0,0,0,0,0,0,0,0,4.92,0,0.76,0,0,0,0,0,0,1,1,1,0,0,0,0,0,18.0,0,2.57,NaN,3a226ea48debc0a7ae9950d5540f2f34
5,0,0,0,0,0,0,0,0,0,0,0,4.190793,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,699,0,0,0,1,0,0,0,0,0,283.223732,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0.8946,0,0,0,0,0,0,0,0,4.84,0,1.00,0,0,1,0,0,0,1,0,1,0,0,0,0,1,15.0,0,1.82,NaN,9bf29b56619fcaf60b52690a848e10bb
9,0,0,0,0,0,0,0,1,0,0,0,2.512359,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7698,0,0,0,1,0,0,0,0,0,283.223732,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2.852,0,0,0,0,0.9363,0,0,0,0,0,0,0,0,4.84,0,1.00,0,0,1,0,0,0,1,0,1,0,0,0,0,1,94.0,0,1.67,NaN,299d948a5fd2cf2a921894b9bd24b94e


In [20]:
ofs.isnull().sum()

city_origin_La Serena                            0
city_origin_Conchalí                             0
Hour_1                                           0
city_origin_Viña del Mar                         0
city_origin_Colina                               0
Hour_3                                           0
city_destiny_San Pedro de la Paz                 0
Hour_21                                          0
city_destiny_La Serena                           0
city_origin_Valparaíso                           0
Hour_19                                          0
distance_havesine                                0
city_origin_Las Condes                           0
city_destiny_undefined                           0
city_origin_Vitacura                             0
city_origin_Recoleta                             0
city_origin_Concepción                           0
city_origin_Coquimbo                             0
city_destiny_La Granja                           0
city_destiny_Lo Espejo         

### Modeling data

In [21]:
modeling = full[~full.total_minutes.isnull()]

In [22]:
modeling.head(3)

,city_origin_La Serena,city_origin_Conchalí,Hour_1,city_origin_Viña del Mar,city_origin_Colina,Hour_3,city_destiny_San Pedro de la Paz,Hour_21,city_destiny_La Serena,city_origin_Valparaíso,Hour_19,distance_havesine,city_origin_Las Condes,city_destiny_undefined,city_origin_Vitacura,city_origin_Recoleta,city_origin_Concepción,city_origin_Coquimbo,city_destiny_La Granja,city_destiny_Lo Espejo,city_origin_San Pedro de la Paz,city_destiny_Vitacura,county_destiny_Provincia de Maipo,city_destiny_Providencia,Hour_15,seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f,county_destiny_Provincia de Cordillera,city_destiny_Quinta Normal,city_destiny_San Ramón,city_destiny_Talcahuano,city_destiny_Lo Prado,Hour_2,city_origin_undefined,city_destiny_Renca,city_destiny_La Pintana,Hour_23,city_origin_Huechuraba,city_destiny_La Florida,shoppers_number,Day_of_Week_5,city_origin_Santiago,Hour_12,county_origin_Provincia de Santiago,seniority_50e13ee63f086c2fe84229348bc91b5b,city_origin_Estación Central,Day_of_Week_6,city_destiny_Cerro Navia,city_origin_Ñuñoa,temperature,seniority_6c90661e6d2c7579f5ce337c3391dbb9,city_origin_San Miguel,county_origin_Provincia de Cordillera,city_destiny_Concepción,Hour_17,on_demand_True,Hour_13,city_destiny_Macul,city_destiny_Coquimbo,city_destiny_Peñalolén,city_origin_Providencia,city_origin_Pudahuel,city_origin_Peñalolén,Hour_20,quantity_KG,city_destiny_San Joaquín,city_destiny_San Miguel,city_destiny_Las Condes,Hour_22,found_rate,city_destiny_Pudahuel,city_destiny_Ñuñoa,city_origin_Talcahuano,city_destiny_Estación Central,city_destiny_Huechuraba,city_destiny_Recoleta,Hour_18,city_destiny_Santiago,rating,city_destiny_Conchalí,accepted_rate,Hour_11,period_dawn,city_destiny_Lo Barnechea,city_destiny_El Bosque,city_destiny_La Cisterna,city_destiny_Valparaíso,same_county_1.0,Hour_14,same_city_1.0,Hour_16,same_neighbourhood_1.0,city_origin_Macul,city_origin_San Joaquín,period_night,quantity_UN,city_destiny_Independencia,picking_speed,total_minutes,order_id
0,0,0,0,0,0,0,0,0,0,0,0,1.823597,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7698,0,0,0,1,0,0,0,0,0,283.223732,1,0,0,0,0,1,0,0,0,1,0,0,0,1,2.756,0,0,0,0,0.9024,0,0,0,0,0,0,0,0,4.76,0,0.92,0,0,0,0,0,0,1,0,0,0,0,1,0,1,16.0,0,1.30,67.684264,e750294655c2c7c34d83cc3181c09de4
1,0,0,1,0,0,0,0,0,0,0,0,1.935026,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,7698,1,0,0,1,0,0,0,0,0,282.664238,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0.7610,0,0,0,0,0,0,0,0,4.96,0,0.92,0,1,0,0,0,0,1,0,0,0,1,0,0,0,11.0,0,2.54,57.060632,6581174846221cb6c467348e87f57641
3,0,0,0,0,0,0,0,1,0,0,0,3.820244,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,0,0,0,1,0,0,0,0,0,283.223732,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0.8776,0,0,0,0,0,0,0,0,4.76,0,0.96,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1.0,0,2.80,52.067742,7d2ed03fe4966083e74b12694b1669d8


In [23]:
modeling.isnull().sum()

city_origin_La Serena                         0
city_origin_Conchalí                          0
Hour_1                                        0
city_origin_Viña del Mar                      0
city_origin_Colina                            0
Hour_3                                        0
city_destiny_San Pedro de la Paz              0
Hour_21                                       0
city_destiny_La Serena                        0
city_origin_Valparaíso                        0
Hour_19                                       0
distance_havesine                             0
city_origin_Las Condes                        0
city_destiny_undefined                        0
city_origin_Vitacura                          0
city_origin_Recoleta                          0
city_origin_Concepción                        0
city_origin_Coquimbo                          0
city_destiny_La Granja                        0
city_destiny_Lo Espejo                        0
city_origin_San Pedro de la Paz         

In [24]:
# Getting the data:
X = modeling.drop(['total_minutes', 'order_id'], axis = 1)

y = modeling['total_minutes']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 451, test_size= .25)

In [26]:
StdSca = StandardScaler()  
X_train[var_num] = pd.DataFrame(StdSca.fit_transform(X_train[var_num]), columns = var_num, index = X_train.index)
X_test[var_num] = pd.DataFrame(StdSca.transform(X_test[var_num]), columns = var_num, index = X_test.index)

C:\Users\aureliano.paiva_tc\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\aureliano.paiva_tc\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [27]:
ofs[var_num] = StdSca.transform(ofs[var_num])

C:\Users\aureliano.paiva_tc\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


## Model
    I tried to use another models but the ElasticNet was the one that got a better result.

In [28]:
CV = cross_validate(estimator = ElasticNet(alpha=0.025, l1_ratio=0.5), 
                    cv = 10, 
                    scoring = ["r2", 'neg_median_absolute_error', 'max_error', 'neg_mean_squared_error'], 
                    return_estimator = True,
                    return_train_score= False,
                    X = X_train,
                    y = y_train)
cv_df = pd.DataFrame(CV)
cv_df

,fit_time,score_time,estimator,test_r2,test_neg_median_absolute_error,test_max_error,test_neg_mean_squared_error
0,0.581608,0.002998,ElasticNet(alpha=0.025),0.456725,-14.397423,-143.850533,-622.621304
1,0.004097,0.017762,ElasticNet(alpha=0.025),0.453722,-14.305869,-183.920286,-651.606138
2,0.023337,0.001998,ElasticNet(alpha=0.025),0.485985,-14.753840,-114.078430,-538.691464
3,0.023556,0.002000,ElasticNet(alpha=0.025),0.459448,-15.194936,-128.652171,-616.124889
4,0.024256,0.003000,ElasticNet(alpha=0.025),0.456282,-14.743981,-136.695522,-615.132630
5,0.039462,0.004364,ElasticNet(alpha=0.025),0.476077,-14.079874,-89.813774,-557.092240
6,0.039618,0.006995,ElasticNet(alpha=0.025),0.510836,-14.125882,-187.945318,-599.048613
7,0.029332,0.016012,ElasticNet(alpha=0.025),0.498695,-15.433240,-187.441914,-609.969372
8,0.024986,0.004895,ElasticNet(alpha=0.025),0.484903,-14.034609,-143.685824,-587.366473
9,0.015783,0.000000,ElasticNet(alpha=0.025),0.473732,-15.058256,-114.895262,-519.965439


In [29]:
CV = CV["estimator"]

    I chose the model based on the max_error.

In [30]:
cv_df['test_max_error'] = abs(cv_df['test_max_error'])
index = cv_df['fit_time'][cv_df.test_max_error == min(cv_df.test_max_error)].index.values[0]
index

5

In [31]:
X_ofs = ofs.drop(['total_minutes', 'order_id'], axis = 1)
ofs['prediction'] = CV[index].predict(X_ofs)
ofs = ofs[['order_id','prediction']]
ofs.to_csv(os.path.join(path0, "submitted.csv"))

C:\Users\AURELI~1.PAI\AppData\Local\Temp/ipykernel_17240/3913602559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofs['prediction'] = CV[index].predict(X_ofs)


In [32]:
#Fim